#**Autoencoder Model for Word Embedding**

#I. Import necessary things

In [1]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
import re
import os
import pickle
import nltk
import numpy as np
from scipy.spatial import distance

nltk.download('stopwords')
nltk.download('punkt')

#II. Load corpus and preprocess it

In [2]:
# Load data
def load_data(csv_file):
    return pd.read_csv(csv_file, encoding="latin")
    
train_data = load_data("datasets/Corona_NLP_train.csv").iloc[:2000]

corpus = train_data["OriginalTweet"].to_list()
corpus = "\n".join(corpus)
print(corpus[:1000])

@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/iFz9FAn2Pa and https://t.co/xX6ghGFzCC and https://t.co/I2NlzdxNo8
advice Talk to your neighbours family to exchange phone numbers create contact list with phone numbers of neighbours schools employer chemist GP set up online shopping accounts if poss adequate supplies of regular meds but not over order
Coronavirus Australia: Woolworths to give elderly, disabled dedicated shopping hours amid COVID-19 outbreak https://t.co/bInCA9Vp8P
My food stock is not the only one which is empty...



PLEASE, don't panic, THERE WILL BE ENOUGH FOOD FOR EVERYONE if you do not take more than you need. 

Stay calm, stay safe.



#COVID19france #COVID_19 #COVID19 #coronavirus #confinement #Confinementotal #ConfinementGeneral https://t.co/zrlG0Z520j
Me, ready to go at supermarket during the #COVID19 outbreak.



Not because I'm paranoid, but because my food stock is litteraly empty. The #coronavirus is a serious thing, but please, don't panic. It causes shortage

In [3]:
def text_cleaner(text):
    # lower case text
    text = text.lower()
    text = re.sub(r"'s\b","",text)
    # remove punctuations
    text = re.sub("[^a-zA-Z]", " ", text)
    return text

In [4]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

def preprocess_text(corpus):
  corpus_clean = text_cleaner(corpus)

  stop_words = set(stopwords.words('english'))
  word_tokens = word_tokenize(corpus_clean)
  
  return [w for w in word_tokens if not w in stop_words]

corpus_token =  preprocess_text(corpus)
print(corpus_token)

['menyrbie', 'phil', 'gahan', 'chrisitv', 'https', 'co', 'ifz', 'fan', 'pa', 'https', 'co', 'xx', 'ghgfzcc', 'https', 'co', 'nlzdxno', 'advice', 'talk', 'neighbours', 'family', 'exchange', 'phone', 'numbers', 'create', 'contact', 'list', 'phone', 'numbers', 'neighbours', 'schools', 'employer', 'chemist', 'gp', 'set', 'online', 'shopping', 'accounts', 'poss', 'adequate', 'supplies', 'regular', 'meds', 'order', 'coronavirus', 'australia', 'woolworths', 'give', 'elderly', 'disabled', 'dedicated', 'shopping', 'hours', 'amid', 'covid', 'outbreak', 'https', 'co', 'binca', 'vp', 'p', 'food', 'stock', 'one', 'empty', 'please', 'panic', 'enough', 'food', 'everyone', 'take', 'need', 'stay', 'calm', 'stay', 'safe', 'covid', 'france', 'covid', 'covid', 'coronavirus', 'confinement', 'confinementotal', 'confinementgeneral', 'https', 'co', 'zrlg', 'z', 'j', 'ready', 'go', 'supermarket', 'covid', 'outbreak', 'paranoid', 'food', 'stock', 'litteraly', 'empty', 'coronavirus', 'serious', 'thing', 'please'

In [5]:
dictionary = sorted(list(set(corpus_token)))

if os.path.exists("mapping.pkl"):
  print("Load mapping")
  mapping = pickle.load(open("mapping.pkl", "rb"))
else:
  print("Save mapping")
  mapping = dict((c, i) for i, c in enumerate(dictionary))
  pickle.dump(mapping, open("mapping.pkl", "wb"))

print(dictionary)

Load mapping
['aa', 'aacounty', 'aagbwjm', 'ab', 'abandons', 'abate', 'abating', 'abbott', 'abc', 'abcactionnews', 'abidsuleri', 'abilene', 'ability', 'abl', 'able', 'abled', 'abou', 'abrir', 'abroad', 'abrupt', 'absolute', 'absolutely', 'absurd', 'abt', 'abuelos', 'abundance', 'abundant', 'abused', 'ac', 'aca', 'accc', 'acccgovau', 'accelerant', 'accelerated', 'accelerates', 'accelerating', 'accept', 'acceptable', 'accepting', 'access', 'accessible', 'accidentally', 'accommodate', 'accommodation', 'accommodations', 'according', 'accordingly', 'account', 'accountant', 'accounts', 'accumulation', 'accurate', 'accuse', 'accused', 'acejointhealth', 'aceli', 'ach', 'acquaintance', 'acquired', 'acquiring', 'acquisti', 'across', 'act', 'acted', 'acting', 'action', 'actions', 'activated', 'activates', 'activating', 'active', 'actively', 'activities', 'activity', 'actsofkindness', 'actual', 'actually', 'actuallyautistic', 'ad', 'adapt', 'adapting', 'adara', 'add', 'added', 'addiction', 'adding

#III. Convert corpus to one-hot vectors & Define embedding dim

In [6]:
embedding_dim = 256
vocab_size = len(dictionary)
print(vocab_size)

9382


In [7]:
corpus_encode = [mapping[x] for x in corpus_token]
onehot_corpus = keras.utils.to_categorical(corpus_encode, num_classes=vocab_size)
print(onehot_corpus)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


#IV. Define Autoencoder model

In [8]:
ae_model = keras.Sequential()
ae_model.add(keras.Input(shape=(vocab_size,)))
ae_model.add(keras.layers.Dense(embedding_dim, activation='relu'))
ae_model.add(keras.layers.Dense(vocab_size, activation='softmax'))

ae_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])

if os.path.exists("ae_model.h5"):
  ae_model.load_weights("ae_model.h5")

ae_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               2402048   
                                                                 
 dense_1 (Dense)             (None, 9382)              2411174   
                                                                 
Total params: 4813222 (18.36 MB)
Trainable params: 4813222 (18.36 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


#V. Train AE model

In [9]:
# Train model
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("ae_model.h5", monitor='val_loss', save_best_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

ae_model.fit(x=onehot_corpus, y=onehot_corpus, batch_size=512, epochs=1000,
          validation_split=0.2, validation_freq=1, shuffle=True,
          callbacks=[checkpoint_cb, early_stopping_cb],
          workers=1, use_multiprocessing=False)

ae_model.save("ae_model.h5")

Epoch 1/1000
66/66 [==============================] - 8s 117ms/step - loss: 8.7745 - accuracy: 0.2350 - val_loss: 7.8803 - val_accuracy: 0.0923
Epoch 2/1000
 1/66 [..............................] - ETA: 6s - loss: 7.7455 - accuracy: 0.1016

c:\Users\TheNM\Downloads\Coding\School\NLP\venv\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


66/66 [==============================] - 7s 110ms/step - loss: 7.0714 - accuracy: 0.0695 - val_loss: 6.9761 - val_accuracy: 0.0797
Epoch 3/1000
66/66 [==============================] - 7s 108ms/step - loss: 6.3284 - accuracy: 0.1239 - val_loss: 6.3364 - val_accuracy: 0.2029
Epoch 4/1000
66/66 [==============================] - 7s 109ms/step - loss: 5.4179 - accuracy: 0.2995 - val_loss: 5.4953 - val_accuracy: 0.4061
Epoch 5/1000
66/66 [==============================] - 7s 107ms/step - loss: 4.4122 - accuracy: 0.5074 - val_loss: 4.7524 - val_accuracy: 0.5657
Epoch 6/1000
66/66 [==============================] - 7s 108ms/step - loss: 3.6033 - accuracy: 0.6264 - val_loss: 4.2651 - val_accuracy: 0.6317
Epoch 7/1000
66/66 [==============================] - 7s 106ms/step - loss: 3.0644 - accuracy: 0.6855 - val_loss: 3.9738 - val_accuracy: 0.6659
Epoch 8/1000
66/66 [==============================] - 7s 105ms/step - loss: 2.7042 - accuracy: 0.7215 - val_loss: 3.7818 - val_accuracy: 0.6885
Epoch

#=======================================================================

# **How to use trained model**

#=======================================================================

#VI. Load trained model & Get output of 1st FC layer

In [ ]:
reconstructed_model = keras.models.load_model("ae_model.h5")
we_model = keras.models.Model(inputs=reconstructed_model.inputs, outputs=reconstructed_model.get_layer('dense').output)

Define function to encode one-hot list of words

In [ ]:
def encode_onehot(mapping, list_words):
  output = []
  
  for word in list_words:
    word_vector = [0 for _ in range(vocab_size)]

    if word in mapping.keys():
      word_index = mapping[word]
      word_vector[word_index] = 1
    
    output.append(word_vector)
  
  return output

#VII. Test model

In [ ]:
input_sentence = "Covid is very dangerous"

preprocess_sentence = preprocess_text(input_sentence)
onehot_sentence = encode_onehot(mapping, preprocess_sentence)

embedded_sentence = we_model.predict(onehot_sentence)
print(embedded_sentence)

In [ ]:
word_0 = "virus"
word_1 = "vaccines"
word_2 = "playing"

preprocess_words = preprocess_text(' '.join([word_0, word_1, word_2]))
onehot_words = encode_onehot(mapping, preprocess_words)

word_0_eb, word_1_eb, word_2_eb = we_model.predict(onehot_words)

# print("OUTPUT EMBEDDING")
# print(word_0_eb)
# print(word_1_eb)
# print(word_2_eb)

dst_0_1 = distance.euclidean(word_0_eb, word_1_eb)
dst_1_2 = distance.euclidean(word_1_eb, word_2_eb)
dst_2_0 = distance.euclidean(word_2_eb, word_0_eb)

print("OUTPUT DISTANCE")
print("0 vs 1: ", dst_0_1)
print("1 vs 2: ", dst_1_2)
print("2 vs 0: ", dst_2_0)

In [ ]:
from scipy.spatial import distance_matrix
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt

list_words = ["virus", "corona", "hotel", "covid", "breath", "car", "immune", "vaccines", "money", "playing"]
onehot_words = encode_onehot(mapping, list_words)
list_word_embs = we_model.predict(onehot_words)

dm = distance_matrix(list_word_embs, list_word_embs, p=2)

df_dm = pd.DataFrame(dm, list_words, list_words)
plt.figure(figsize=(10,7))
sn.set(font_scale=1.4) # for label size
sn.heatmap(df_dm, annot=True, annot_kws={"size": 16}) # font size

plt.show()